<a href="https://colab.research.google.com/github/aksh98/Attribute_Classification/blob/master/1Nov_MorphingUsingGANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torch
!pip3 install torchvision


    100% |████████████████████████████████| 592.3MB 33.9MB/s 
    100% |████████████████████████████████| 61kB 2.6MB/s 
    100% |████████████████████████████████| 2.0MB 7.5MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/My Drive/Colab Notebooks
# %cd Colab Notebooks
# %cd Gans_out
# !ls
# !mv 'Models (1)' Models
# %cd ..
# !ls
# !ls | wc -l
# %cd ..

/content/drive/My Drive/Colab Notebooks


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [5]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import scipy.io
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import Normalize
import os
import pickle
from torchvision import datasets, transforms
import torchvision.utils as vutil
print("Done")

Done


In [0]:
from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

In [0]:
from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

mini_batch_size = 16
input_size = 64

to_out = []

def read_data(folder):
  count = 0
  dataset = []
  for imgfile in os.listdir(folder):
    img_data = cv2.imread(os.path.join(folder,imgfile))
    if(img_data is not None):
      img_data = cv2.resize(img_data,(64,64))
      dataset.append(image_data)
      count += 1
    dataset = np.array(dataset)
    return dataset
  
########################################################

class RealAB(nn.Module):
  
  def __init__(self,in_channels):
    super(RealAB, self).__init__()
    #image1
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels,8,kernel_size = 5,stride=1,padding=2),
        nn.BatchNorm2d(8),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride = 2))
    self.layer2 = nn.Sequential(
        nn.Conv2d(8,16,kernel_size = 5,stride = 1,padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc = nn.Linear(16*16*16,64)
    
    #image2
    self.layer11 = nn.Sequential(
        nn.Conv2d(in_channels,8,kernel_size = 5,stride=1,padding=2),
        nn.BatchNorm2d(8),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride = 2))
    self.layer21 = nn.Sequential(
        nn.Conv2d(8,16,kernel_size = 5,stride = 1,padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc1 = nn.Linear(16*16*16,64)
    
  def forward(self,x, y):
#     print(x.shape,y.shape)
    out = self.layer1(x)
    out = self.layer2(out)
#     print(out.shape)
    out = out.view(out.size(0),-1)
    out = self.fc(out)
    
    out1 = self.layer11(y)
    out1 = self.layer21(out1)
#     print("out1sh-",out1.shape)  # 16,16,16,16
##### batch_size, channels_out * height_out * width_out
    out1 = out1.view(out1.size(0),-1) 
    out1 = self.fc1(out1)
#     print("out1",out1.shape,out.shape)  # 16,64 
    final = torch.cat((out,out1),1)
#     print("fial",final.shape) #16*128
    return final
    
    
    
########################################################
  
class generator(nn.Module):
  
  def __init__(self, d = 16):
    super(generator,self).__init__()
    # in_channels, out_channels, kernel_size, stride=1, padding=0
    self.inpAB = RealAB(in_channels = 3)
    self.deconv1 = nn.ConvTranspose2d(128, d*8, 4, 1, 0)
    self.deconv1_bn = nn.BatchNorm2d(d*8)
    self.deconv2 = nn.ConvTranspose2d(d*8, d*4, 4, 2, 1)
    self.deconv2_bn = nn.BatchNorm2d(d*4)
    self.deconv3 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
    self.deconv3_bn = nn.BatchNorm2d(d*2)
    self.deconv4 = nn.ConvTranspose2d(d*2, d  , 4, 2, 1)
    self.deconv4_bn = nn.BatchNorm2d(d)
    self.deconv5 = nn.ConvTranspose2d(d  , 3  , 4, 2, 1)
    
    for i in self.modules():
      if(isinstance(i, nn.ConvTranspose2d)):
        i.weight.data.normal_(0.0,0.2)
        if(i.bias is not None):
          i.bias.data.zero_()
          
  def forward(self,input1,input2): 
    x = self.inpAB(input1,input2)
#     print("--->",x.shape)
    x = x.view(x.size(0),128,1,1)
#     print("syappa -",x.size())
    x = F.relu(self.deconv1_bn(self.deconv1(x)))
    x = F.relu(self.deconv2_bn(self.deconv2(x)))
    x = F.relu(self.deconv3_bn(self.deconv3(x)))
    x = F.relu(self.deconv4_bn(self.deconv4(x)))
    x = F.tanh(self.deconv5(x))
    return x
  
class discriminator(nn.Module):
  def __init__(self, d = 16):
    super(discriminator,self).__init__()
    
    self.conv1 = nn.Conv2d(3, d, 4, 2 , 1)
    self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1)
    self.conv2_bn = nn.BatchNorm2d(d*2)
    self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
    self.conv3_bn = nn.BatchNorm2d(d*4)
    self.conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1)
    self.conv4_bn = nn.BatchNorm2d(d*8)
    self.conv5 = nn.Conv2d(d*8, 1, 4, 1, 0)
    
    for i in self.modules():
      if( isinstance(i,nn.ConvTranspose2d)):
        i.weight.data.normal_(0.0, 0.2)
        if(i.bias is not None):
          i.bias,data.zero_()
          
  def forward(self,input):
    x = F.leaky_relu(self.conv1(input),0.2)
    x = F.leaky_relu(self.conv2_bn(self.conv2(x)),0.2)
    x = F.leaky_relu(self.conv3_bn(self.conv3(x)),0.2)
    x = F.leaky_relu(self.conv4_bn(self.conv4(x)),0.2)
    x = F.sigmoid(self.conv5(x))
    
    return x

##############################################################
  
# model = TheModelClass(*args, **kwargs)
# optimizer = TheOptimizerClass(*args, **kwargs)
# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

# model.eval()
  
##############################################################  
  
generator = generator(16)
discriminator = discriminator(16)
generator.cuda
discriminator.cuda

# criterionL1 = nn.L1Loss()
#criterionMSE = nn.MSELoss()
#criterionGAN = GANLoss()
#criterionL1 = criterionL1.cuda()
#criterionMSE = criterionMSE.cuda()
#criterionGAN = criterionGAN.cuda()

discriminator_optimizer = optim.Adam( discriminator.parameters(),lr = 0.0002)
generator_optimizer = optim.Adam(generator.parameters(),lr = 0.0002)

checkpointG = torch.load('/content/drive/My Drive/Colab Notebooks/Gans_out/Models/G.pth.tar') #
checkpointD = torch.load('/content/drive/My Drive/Colab Notebooks/Gans_out/Models/D.pth.tar') #

generator.load_state_dict(checkpointG['state_dict'])
discriminator.load_state_dict(checkpointD['state_dict'])

generator_optimizer.load_state_dict(checkpointG['optimizer_state_dict'])
discriminator_optimizer.load_state_dict(checkpointD['optimizer_state_dict'])

epoch = checkpointG['epoch']
loss_G = checkpointG['loss']
loss_D = checkpointD['loss']

generator.eval()
discriminator.eval()

# epoch = 0
# loss_D = []
# loss_G = []


criterion = nn.BCELoss()
# criterion = criterion.cuda


img_size = 64

transform = transforms.Compose([ transforms.Resize((64,64)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5,0.5,0.5),std = (0.5,0.5,0.5))
                              ])
data_dir = '/content/drive/My Drive/Colab Notebooks/BiomProject/swapped/'
dset = datasets.ImageFolder(data_dir,transform)
train_loader = torch.utils.data.DataLoader(dset, batch_size=mini_batch_size, shuffle=True)

gen_data = '/content/drive/My Drive/Colab Notebooks/lfw_aligned'
genset = datasets.ImageFolder(gen_data,transform)
gentrain_loader = torch.utils.data.DataLoader(genset, batch_size=mini_batch_size,shuffle=True)
gentrain_loader2 = torch.utils.data.DataLoader(genset, batch_size=mini_batch_size,shuffle=True)

print("Data Loaded")

exit()
# print("Exitkebaad")
c = 0
generator.zero_grad()

################YAHA TK SAME HAI ######################

#inputss for generator - swapped gndi images

# train_target = torch.tensor(train['Target'].values)
# input_for_generator =  Variable(gentrain_loader)
# input_for_generator2 =  Variable(gentrain_loader2)

# samples = generator(input_for_generator,input_for_generator2)
print("Hurrah!! ")
# samples = generator(gentrain_loader, gentrain_loader2)

norm = Normalize()

# fig = plt.figure(figsize = (4, 4))

# gs = gridspec.GridSpec(4, 4)
# gs.update(wspace=0.05, hspace=0.05)
# for i in range(16):
#   ax = plt.subplot(gs[i])
#   plt.axis('off')
#   ax.set_xticklabels([])
#   ax.set_yticklabels([])
#   ax.set_aspect('equal')
#   plt.imshow((samples[i].cpu().data.numpy().transpose(1,2,0)+1)/2)
# if (not os.path.exists('drive/My Drive/Colab Notebooks/Gans_out/output1')): # where to store outputs
#   os.makedirs('drive/Colab Notebooks/Gans_out/output1')
# plt.savefig('drive/My Drive/Colab Notebooks/Gans_out/output1/{}.png'.format(str(c),zfill(3)), bbox_inches = 'tight')
# plt.close(fig)
# c +=1
# for x,_ in train_loader:
#   pass
# for i,_ in gentrain_loader:
#   pass
##############################################################
#            Main algo - step1 
##############################################################  


for k in range(epoch,500):
  discriminator.zero_grad()
  print("k -> ",k)
  for x_,y_,z_,in zip(train_loader,gentrain_loader,gentrain_loader2):
    data = Variable(x_[0])
    input_real = data
    discriminator_real_out = discriminator(input_real)
    # maybe a problem
    discriminator_real_error = criterion(discriminator_real_out, Variable(torch.ones(mini_batch_size,1)))
    
    #send in the input ----
    input_for_generator = Variable(y_[0])
    input_for_generator2 = Variable(z_[0])
    generator_gen_data = generator(input_for_generator,input_for_generator2).detach()
    discriminator_fake_out = discriminator(generator_gen_data)
    discriminator_fake_out = discriminator_fake_out.view(discriminator_fake_out.size(0),-1)
    print("lol",discriminator_fake_out.size(),Variable(torch.zeros(mini_batch_size,1)).size())
    discriminator_fake_error = criterion(discriminator_fake_out,Variable(torch.zeros(mini_batch_size,1)))
    
    discriminator_total_error = discriminator_fake_error + discriminator_real_error
    discriminator_total_error.backward()
    discriminator_optimizer.step()
    
    # STEP 2 _ TRAIN GENERATOR 
    generator.zero_grad()
    # send in the input -------
    input_for_generator = Variable(y_[0])
    input_for_generator2 = Variable(z_[0])
    generator_out = generator(input_for_generator,input_for_generator2)
    discriminator_out = discriminator(generator_out)
    generator_error = criterion(discriminator_out, Variable(torch.ones(mini_batch_size,1)))
    print(".",end='')
    generator_error.backward()
    generator_optimizer.step()
  print('')  
  print('Iteration{}  Disc_loss:{}, Gen_Loss: {}'.format(k,discriminator_total_error.data.numpy(),generator_error.data.numpy()))
  loss_D.append(discriminator_total_error.data.numpy())
  loss_G.append(generator_error.data.numpy())

  torch.save({
      'epoch':k, 
      'state_dict': generator.state_dict(),
      'optimizer_state_dict': generator_optimizer.state_dict(),
      'loss': loss_G, 
    },'/content/drive/My Drive/Colab Notebooks/Gans_out/Models/G.pth.tar')
  torch.save({
      'epoch': k, 
      'state_dict': discriminator.state_dict(),
      'optimizer_state_dict': discriminator_optimizer.state_dict(),
      'loss': loss_D,
    },'/content/drive/My Drive/Colab Notebooks/Gans_out/Models/D.pth.tar')
  print("Output stored")
 
##################################8############################
#            Train the generator
##############################################################  

  # send the input again
  for y_,z_ in zip(gentrain_loader,gentrain_loader2):
    print("e-",end='')
    input_for_generator = Variable(y_[0])
    input_for_generator2 = Variable(z_[0])
    samples = generator(input_for_generator,input_for_generator2)
    norm = Normalize()
    fig = plt.figure(figsize=(4,4))
    gs = gridspec.GridSpec(4,4)
    gs.update(wspace=0.05,hspace = 0.05)
  
    for i in range(16):
      ax = plt.subplot(gs[i])
      plt.axis('off')
      ax.set_xticklabels([])
      ax.set_yticklabels([])
      ax.set_aspect('equal')
      plt.imshow((samples[i].cpu().data.numpy().transpose(1, 2, 0)+1)/2)
    plt.savefig('./Gans_out/output2/{}.png'.format(str(c).zfill(3)), bbox_inches='tight')
    c += 1
    plt.close(fig)
    
#     input_for_generator = Variable(input_for_generator,)
    samples = generator(input_for_generator,input_for_generator2).data.numpy()[:1]
    norm = Normalize()
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(1, 1)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(norm(samples[0].transpose((1, 2, 0))))
    if not os.path.exists('./Gans_out/output1/'):
        os.makedirs('./Gans_out/output1/')

    plt.savefig('./Gans_out/output1/{}.png'.format(str(c).zfill(3)), bbox_inches='tight')
    c += 1
    plt.close(fig)

arr = [loss_D,loss_G]
pickle.dump(to_out,open("./Gans_out/Models/loss.dat","wb"))
print("HAHAHAHAHAHHA")

  

In [0]:
!pwd

## Example 

In [0]:

# import torch 
# import torch.nn as nn
# import torchvision
# import torchvision.transforms as transforms


# # Device configuration
# # device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# # Hyper parameters
# num_epochs = 5
# num_classes = 10
# batch_size = 100
# learning_rate = 0.001

# # MNIST dataset
# train_dataset = torchvision.datasets.MNIST(root='../../data/',
#                                            train=True, 
#                                            transform=transforms.ToTensor(),
#                                            download=True)

# test_dataset = torchvision.datasets.MNIST(root='../../data/',
#                                           train=False, 
#                                           transform=transforms.ToTensor())

# # Data loader
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                            batch_size=batch_size, 
#                                            shuffle=True)

# test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                           batch_size=batch_size, 
#                                           shuffle=False)

# # Convolutional neural network (two convolutional layers)
# class ConvNet(nn.Module):
#     def __init__(self, num_classes=10):
#         super(ConvNet, self).__init__()
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
#             nn.BatchNorm2d(16),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2))
#         self.layer2 = nn.Sequential(
#             nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2))
#         self.fc = nn.Linear(7*7*32, num_classes)
        
#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = out.reshape(out.size(0), -1)
#         out = self.fc(out)
#         return out

# model = ConvNet(num_classes)

# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # Train the model
# total_step = len(train_loader)
# for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(train_loader):
#         images = images
#         labels = labels
        
#         # Forward pass
#         outputs = model(images)
#         loss = criterion(outputs, labels)
        
#         # Backward and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         if (i+1) % 100 == 0:
#             print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
#                    .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# # Test the model
# model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images
#         labels = labels
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# # Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')


In [0]:
for x in range(10,250,10):
  print(x)

## The backup

In [0]:
mini_batch_size = 16
input_size = 64

to_out = []

def read_data(folder):
  count = 0
  dataset = []
  for imgfile in os.listdir(folder):
    img_data = cv2.imread(os.path.join(folder,imgfile))
    if(img_data is not None):
      img_data = cv2.resize(img_data,(64,64))
      dataset.append(image_data)
      count += 1
    dataset = np.array(dataset)
    return dataset
  
########################################################

class RealAB(nn.Module):
  
  def __init__(self,in_channels):
    super(RealAB, self).__init__()
    #image1
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels,8,kernel_size = 5,stride=1,padding=2),
        nn.BatchNorm2d(8),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride = 2))
    self.layer2 = nn.Sequential(
        nn.Conv2d(8,16,kernel_size = 5,stride = 1,padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc = nn.Linear(16*16*16,64)
    
    #image2
    self.layer11 = nn.Sequential(
        nn.Conv2d(in_channels,8,kernel_size = 5,stride=1,padding=2),
        nn.BatchNorm2d(8),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride = 2))
    self.layer21 = nn.Sequential(
        nn.Conv2d(8,16,kernel_size = 5,stride = 1,padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc1 = nn.Linear(16*16*16,64)
    
  def forward(self,x, y):
#     print(x.shape,y.shape)
    out = self.layer1(x)
    out = self.layer2(out)
#     print(out.shape)
    out = out.view(out.size(0),-1)
    out = self.fc(out)
    
    out1 = self.layer11(y)
    out1 = self.layer21(out1)
#     print("out1sh-",out1.shape)  # 16,16,16,16
##### batch_size, channels_out * height_out * width_out
    out1 = out1.view(out1.size(0),-1) 
    out1 = self.fc1(out1)
#     print("out1",out1.shape,out.shape)  # 16,64 
    final = torch.cat((out,out1),1)
#     print("fial",final.shape) #16*128
    return final
    
    
    
########################################################
  
class generator(nn.Module):
  
  def __init__(self, d = 16):
    super(generator,self).__init__()
    # in_channels, out_channels, kernel_size, stride=1, padding=0
    self.inpAB = RealAB(in_channels = 3)
    self.deconv1 = nn.ConvTranspose2d(128, d*8, 4, 1, 0)
    self.deconv1_bn = nn.BatchNorm2d(d*8)
    self.deconv2 = nn.ConvTranspose2d(d*8, d*4, 4, 2, 1)
    self.deconv2_bn = nn.BatchNorm2d(d*4)
    self.deconv3 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
    self.deconv3_bn = nn.BatchNorm2d(d*2)
    self.deconv4 = nn.ConvTranspose2d(d*2, d  , 4, 2, 1)
    self.deconv4_bn = nn.BatchNorm2d(d)
    self.deconv5 = nn.ConvTranspose2d(d  , 3  , 4, 2, 1)
    
    for i in self.modules():
      if(isinstance(i, nn.ConvTranspose2d)):
        i.weight.data.normal_(0.0,0.2)
        if(i.bias is not None):
          i.bias.data.zero_()
          
  def forward(self,input1,input2): 
    x = self.inpAB(input1,input2)
#     print("--->",x.shape)
    x = x.view(x.size(0),128,1,1)
#     print("syappa -",x.size())
    x = F.relu(self.deconv1_bn(self.deconv1(x)))
    x = F.relu(self.deconv2_bn(self.deconv2(x)))
    x = F.relu(self.deconv3_bn(self.deconv3(x)))
    x = F.relu(self.deconv4_bn(self.deconv4(x)))
    x = F.tanh(self.deconv5(x))
    return x
  
class discriminator(nn.Module):
  def __init__(self, d = 16):
    super(discriminator,self).__init__()
    
    self.conv1 = nn.Conv2d(3, d, 4, 2 , 1)
    self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1)
    self.conv2_bn = nn.BatchNorm2d(d*2)
    self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
    self.conv3_bn = nn.BatchNorm2d(d*4)
    self.conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1)
    self.conv4_bn = nn.BatchNorm2d(d*8)
    self.conv5 = nn.Conv2d(d*8, 1, 4, 1, 0)
    
    for i in self.modules():
      if( isinstance(i,nn.ConvTranspose2d)):
        i.weight.data.normal_(0.0, 0.2)
        if(i.bias is not None):
          i.bias,data.zero_()
          
  def forward(self,input):
    x = F.leaky_relu(self.conv1(input),0.2)
    x = F.leaky_relu(self.conv2_bn(self.conv2(x)),0.2)
    x = F.leaky_relu(self.conv3_bn(self.conv3(x)),0.2)
    x = F.leaky_relu(self.conv4_bn(self.conv4(x)),0.2)
    x = F.sigmoid(self.conv5(x))
    
    return x

##############################################################
  
generator = generator(16)
discriminator = discriminator(16)
generator.cuda
discriminator.cuda

# criterionL1 = nn.L1Loss()
#criterionMSE = nn.MSELoss()
#criterionGAN = GANLoss()

#criterionL1 = criterionL1.cuda()
#criterionMSE = criterionMSE.cuda()
#criterionGAN = criterionGAN.cuda()

discriminator_optimizer = optim.Adam( discriminator.parameters(),lr = 0.0002)
generator_optimizer = optim.Adam(generator.parameters(),lr = 0.0002)
criterion = nn.BCELoss()
# criterion = criterion.cuda


img_size = 64

transform = transforms.Compose([ transforms.Resize((64,64)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5,0.5,0.5),std = (0.5,0.5,0.5))
                              ])
data_dir = '/content/drive/My Drive/Colab Notebooks/BiomProject/swapped/'
dset = datasets.ImageFolder(data_dir,transform)
train_loader = torch.utils.data.DataLoader(dset, batch_size=mini_batch_size, shuffle=True)

gen_data = '/content/drive/My Drive/Colab Notebooks/lfw_aligned'
genset = datasets.ImageFolder(gen_data,transform)
gentrain_loader = torch.utils.data.DataLoader(genset, batch_size=mini_batch_size,shuffle=True)
gentrain_loader2 = torch.utils.data.DataLoader(genset, batch_size=mini_batch_size,shuffle=True)

print("Data Loaded")

exit()
# print("Exitkebaad")
c = 0
generator.zero_grad()

################YAHA TK SAME HAI ######################

#inputss for generator - swapped gndi images

# train_target = torch.tensor(train['Target'].values)
# input_for_generator =  Variable(gentrain_loader)
# input_for_generator2 =  Variable(gentrain_loader2)

# samples = generator(input_for_generator,input_for_generator2)
print("Hurrah!! ")
# samples = generator(gentrain_loader, gentrain_loader2)

norm = Normalize()

# fig = plt.figure(figsize = (4, 4))

# gs = gridspec.GridSpec(4, 4)
# gs.update(wspace=0.05, hspace=0.05)
# for i in range(16):
#   ax = plt.subplot(gs[i])
#   plt.axis('off')
#   ax.set_xticklabels([])
#   ax.set_yticklabels([])
#   ax.set_aspect('equal')
#   plt.imshow((samples[i].cpu().data.numpy().transpose(1,2,0)+1)/2)
# if (not os.path.exists('drive/My Drive/Colab Notebooks/Gans_out/output1')): # where to store outputs
#   os.makedirs('drive/Colab Notebooks/Gans_out/output1')
# plt.savefig('drive/My Drive/Colab Notebooks/Gans_out/output1/{}.png'.format(str(c),zfill(3)), bbox_inches = 'tight')
# plt.close(fig)
# c +=1

loss_D = []
loss_G = []

# for x,_ in train_loader:
#   pass

# for i,_ in gentrain_loader:
#   pass
##############################################################
#            Main algo - step1 
##############################################################  


for k in range(500):
  discriminator.zero_grad()
  print("k -> ",k)
  for x_,y_,z_,in zip(train_loader,gentrain_loader,gentrain_loader2):
    data = Variable(x_[0])
    input_real = data
    discriminator_real_out = discriminator(input_real)
    # maybe a problem
    discriminator_real_error = criterion(discriminator_real_out, Variable(torch.ones(mini_batch_size,1)))
    
    #send in the input ----
    input_for_generator = Variable(y_[0])
    input_for_generator2 = Variable(z_[0])
    generator_gen_data = generator(input_for_generator,input_for_generator2).detach()
    discriminator_fake_out = discriminator(generator_gen_data)
    discriminator_fake_out = discriminator_fake_out.view(discriminator_fake_out.size(0),-1)
    print("lol",discriminator_fake_out.size(),Variable(torch.zeros(mini_batch_size,1)).size())
    discriminator_fake_error = criterion(discriminator_fake_out,Variable(torch.zeros(mini_batch_size,1)))
    
    discriminator_total_error = discriminator_fake_error + discriminator_real_error
    discriminator_total_error.backward()
    discriminator_optimizer.step()
    
    # STEP 2 _ TRAIN GENERATOR 
    generator.zero_grad()
    # send in the input -------
    input_for_generator = Variable(y_[0])
    input_for_generator2 = Variable(z_[0])
    generator_out = generator(input_for_generator,input_for_generator2)
    discriminator_out = discriminator(generator_out)
    generator_error = criterion(discriminator_out, Variable(torch.ones(mini_batch_size,1)))
    print(".",end='')
    generator_error.backward()
    generator_optimizer.step()
  print('')  
  print('Iteration{}  Disc_loss:{}, Gen_Loss: {}'.format(k,discriminator_total_error.data.numpy(),generator_error.data.numpy()))
  loss_D.append(discriminator_total_error.data.numpy())
  loss_G.append(generator_error.data.numpy())
#   torch.save(generator.state_dict(), "./Gans_out/Models/G_{0:03}.pwf".format(k))
#   torch.save(discriminator.state_dict(), "./Gans_out/Models/D_{0:03}.pwf".format(k))
    
  torch.save({
      'epoch':k, 
      'state_dict': generator.state_dict(),
    },'/content/drive/My Drive/Colab Notebooks/Gans_out/Models/G.pth.tar')
  torch.save({
      'epoch': k, 
      'state_dict': discriminator.state_dict(),
    },'/content/drive/My Drive/Colab Notebooks/Gans_out/Models/D.pth.tar')
  print("Output stored")
 
##################################8############################
#            Train the generator
##############################################################  

  # send the input again
  for y_,z_ in zip(gentrain_loader,gentrain_loader2):
    print("e-",end='')
    input_for_generator = Variable(y_[0])
    input_for_generator2 = Variable(z_[0])
    samples = generator(input_for_generator,input_for_generator2)
    norm = Normalize()
    fig = plt.figure(figsize=(4,4))
    gs = gridspec.GridSpec(4,4)
    gs.update(wspace=0.05,hspace = 0.05)
  
    for i in range(16):
      ax = plt.subplot(gs[i])
      plt.axis('off')
      ax.set_xticklabels([])
      ax.set_yticklabels([])
      ax.set_aspect('equal')
      plt.imshow((samples[i].cpu().data.numpy().transpose(1, 2, 0)+1)/2)
    plt.savefig('./Gans_out/output2/{}.png'.format(str(c).zfill(3)), bbox_inches='tight')
    c += 1
    plt.close(fig)
    
#     input_for_generator = Variable(input_for_generator,)
    samples = generator(input_for_generator,input_for_generator2).data.numpy()[:1]
    norm = Normalize()
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(1, 1)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(norm(samples[0].transpose((1, 2, 0))))
    if not os.path.exists('./Gans_out/output1/'):
        os.makedirs('./Gans_out/output1/')

    plt.savefig('./Gans_out/output1/{}.png'.format(str(c).zfill(3)), bbox_inches='tight')
    c += 1
    plt.close(fig) 

arr = [loss_D,loss_G]
pickle.dump(to_out,open("./Gans_out/Models/loss.dat","wb"))
print("HAHAHAHAHAHHA")

  

In [0]:
mini_batch_size = 128
input_size = 100

to_out = []

def read_data(folder):
  count = 0
  dataset = []
  for imgfile in os.listdir(folder):
    img_data = cv2.imread(os.path.join(folder,imgfile))
    if(img_data is not None):
      img_data = cv2.resize(img_data,(250,250))
      dataset.append(image_data)
      count += 1
    dataset = np.array(dataset)
    return dataset
  
########################################################

class RealAB(nn.Module):
  
  def __init__(self,in_channels):
    super(RealAB, self).__init__()
    
    #image1
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels,16,kernel_size = 5,stride=1,padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride = 2))
    self.layer2 = nn.Sequential(
        nn.Conv2d(16,32,kernel_size = 5,stride = 1,padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc = nn.Linear(7*7*32,100)
    
    #image2
    self.layer11 = nn.Sequential(
        nn.Conv2d(in_channels,16,kernel_size = 5,stride=1,padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride = 2))
    self.layer21 = nn.Sequential(
        nn.Conv2d(16,32,kernel_size = 5,stride = 1,padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc1 = nn.Linear(7*7*32,100)
    
  def forward(self,x, y):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0),-1)
    out = self.fc(out)
    
    out1 = self.layer11(y)
    out1 = self.layer21(out1)
    out1 = out1.reshape(out1.size(0),-1)
    out1 = self.fc1(out1)
    
    final = torch.cat((out,out1),1)
    return final
    
    
    
########################################################
  
class generator(nn.Module):
  
  def __init__(self, d = 128):
    super(generator,self).__init__()
    # in_channels, out_channels, kernel_size, stride=1, padding=0
    self.inpAB = RealAB(in_channels = 3)
    self.deconv1 = nn.ConvTranspose2d(100, d*8, 4, 1, 0)
    self.deconv1_bn = nn.BatchNorm2d(d*8)
    self.deconv2 = nn.ConvTranspose2d(d*8, d*4, 4, 2, 1)
    self.deconv2_bn = nn.BatchNorm2d(d*4)
    self.deconv3 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
    self.deconv3_bn = nn.BatchNorm2d(d*2)
    self.deconv4 = nn.ConvTranspose2d(d*2, d  , 4, 2, 1)
    self.deconv4_bn = nn.BatchNorm2d(d)
    self.deconv5 = nn.ConvTranspose2d(d  , 3  , 4, 2, 1)
    
    for i in self.modules():
      if(isinstance(i, nn.ConvTranspose2d)):
        i.weight.data.normal_(0.0,0.2)
        if(i.bias is not None):
          i.bias.data.zero_()
          
  def forward(self,input1,input2): 
    x = self.inpAB(input1,input2)
    x = F.relu(self.deconv1_bn(self.deconv1(x)))
    x = F.relu(self.deconv2_bn(self.deconv2(x)))
    x = F.relu(self.deconv3_bn(self.deconv3(x)))
    x = F.relu(self.deconv4_bn(self.deconv4(x)))
    x = F.tanh(self.deconv5(x))
    return x
  
class discriminator(nn.Module):
  def __init__(self, d = 128 ):
    super(discriminator,self).__init__()
    
    self.conv1 = nn.Conv2d(3, d, 4, 2 , 1)
    self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1)
    self.conv2_bn = nn.BatchNorm2d(d*2)
    self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
    self.conv3_bn = nn.BatchNorm2d(d*4)
    self.conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1)
    self.conv4_bn = nn.BatchNorm2d(d*8)
    self.conv5 = nn.Conv2d(d*8, 1, 4, 1, 0)
    
    for i in self.modules():
      if( isinstance(i,nn.ConvTranspose2d)):
        i.weight.data.normal_(0.0, 0.2)
        if(i.bias is not None):
          i.bias,data.zero_()
          
  def forward(self,input):
    x = F.leaky_relu(self.conv1(input),0.2)
    x = F.leaky_relu(self.conv2_bn(self.conv2(x)),0.2)
    x = F.leaky_relu(self.conv3_bn(self.conv3(x)),0.2)
    x = F.leaky_relu(self.conv4_bn(self.conv4(x)),0.2)
    x = F.sigmoid(self.conv5(x))
    
    return x

##############################################################
  
generator = generator(128)
discriminator = discriminator(128)
# generator.cuda()
# discriminator.cuda()

criterion = nn.BCELoss()
#criterionL1 = nn.L1Loss()
#criterionMSE = nn.MSELoss()
#criterionGAN = GANLoss()

#criterionL1 = criterionL1.cuda()
#criterionMSE = criterionMSE.cuda()
#criterionGAN = criterionGAN.cuda()

discriminator_optimizer = optim.Adam( discriminator.parameters(),lr = 0.0002)
generator_optimizer = optim.Adam(generator.parameters(),lr = 0.0002)


img_size = 250

transform = transforms.Compose([ transforms.Resize((250,250)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5,0.5,0.5),std = (0.5,0.5,0.5))
                              ])
data_dir = '/content/drive/My Drive/Colab Notebooks/BiomProject/swapped/'
dset = datasets.ImageFolder(data_dir,transform)
train_loader = torch.utils.data.DataLoader(dset, batch_size=128, shuffle=True)

gen_data = '/content/drive/My Drive/Colab Notebooks/lfw_aligned'
genset = datasets.ImageFolder(gen_data,transform)
gentrain_loader = torch.utils.data.DataLoader(genset, batch_size=128,shuffle=True)
gentrain_loader2 = torch.utils.data.DataLoader(genset, batch_size=128,shuffle=True)

print("Data Loaded")

exit()

c = 0
generator.zero_grad()

################
#inputss for generator - swapped gndi images

# train_target = torch.tensor(train['Target'].values)
# input_for_generator =  Variable(gentrain_loader)
# input_for_generator2 =  Variable(gentrain_loader2)

# samples = generator(input_for_generator,input_for_generator2)
print("Hurrah!! ")
# samples = generator(gentrain_loader, gentrain_loader2)

norm = Normalize()

# fig = plt.figure(figsize = (4, 4))

# gs = gridspec.GridSpec(4, 4)
# gs.update(wspace=0.05, hspace=0.05)
print("YOLO !!!! ")
# for i in range(16):
#   ax = plt.subplot(gs[i])
#   plt.axis('off')
#   ax.set_xticklabels([])
#   ax.set_yticklabels([])
#   ax.set_aspect('equal')
#   plt.imshow((samples[i].cpu().data.numpy().transpose(1,2,0)+1)/2)
# if (not os.path.exists('drive/My Drive/Colab Notebooks/Gans_out/output1')): # where to store outputs
#   os.makedirs('drive/Colab Notebooks/Gans_out/output1')
# plt.savefig('drive/My Drive/Colab Notebooks/Gans_out/output1/{}.png'.format(str(c),zfill(3)), bbox_inches = 'tight')
# plt.close(fig)
# c +=1

print("Output stored")
loss_D = []
loss_G = []

# for x,_ in train_loader:
#   pass

# for i,_ in gentrain_loader:
#   pass
##############################################################
#            Main algo - step1 
##############################################################  


for k in range(1):
  discriminator.zero_grad()

  for x_,y_,z_,in zip(train_loader,gentrain_loader,gentrain_loader2):
    data = Variable(x_[0])
    input_real = data
    discriminator_real_out = discriminator(input_real)
    # maybe a problem
    discriminator_real_error = criterion(discriminator_real_out, Variable(torch.ones(mini_batch_size,1)))
    
    #send in the input ----
    input_for_generator = Variable(y_[0])
    input_for_generator2 = Variable(z_[0])
    generator_gen_data = generator(input_for_generator,input_for_generator2).detach()
    discriminator_fake_out = discriminator(generator_gen_data)
    discriminator_fake_error = criterion(discriminator_fake_out,Variable(torch.zeros(mini_batch_size,1)))
    
    discriminator_total_error = discriminator_fake_error + discriminator_real_error
    discriminator_total_error.backward()
    discriminator_optimiser.step()
    
    # STEP 2 _ GENERATE IMAGES
    generator.zero_grad()
    # send in the input -------
    input_for_generator = Variable(y_[0])
    input_for_generator2 = Variable(z_[0])
    generator_out = generator(input_for_generator,input_for_generator2)
    discriminator_out = discriminator(generator_out)
    generator_error = criterion(discriminator_out, Variable(torch.ones(mini_batch_size,1)))

    generator_error.backward()
    generator_optimiser.step()
    
  print('Iteration{}  Disc_loss:{}, Gen_Loss: {}'.format(k,discriminator_total_error.data.numpy(),generator_error.data.numpy()))
  loss_D.append(discriminator_total_error.data.numpy())
  loss_G.append(generator_error.data.numpy())
  
  torch.save({
      'epoch':k, 
      'state_dict': generator.state_dict(),
    },'drive/My Drive/Colab Notebooks/Gans_out/Models/G.pth.tar')
  torch.save({
      'epoch': k, 
      'state_dict': discriminator.state_dict(),
    },'drive/My Drive/Colab Notebooks/Gans_out/Models/D.pth.tar')
  
##############################################################
#            PLOT THE IMAGES
##############################################################  

  # send the input again
  for y_,z_ in zip(gentrain_loader,gentrain_loader2):
    input_for_generator = Veriable(y_)
    input_for_generator2 = Veriable(z_)
    samples = generator(input_for_generator,input_for_generator2)
    norm = Normalize()
    fig = plt.figure(figsize=(4,4))
    gs = gridspec.GridSpec(4,4)
    gs.update(wspace=0.05,hspace = 0.05)
  
#   for i in range(16):
#     ax = plt.subplot(gs[i])
#     plt.axis('off')
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])
#     ax.set_aspect('equal')
#     plt.imshow((samples[i].cpu().data.numpy().transpose(1, 2, 0)+1)/2)
#   if not os.path.exists('drive/My Drive/Colab Notebooks/Gans_out/output2/'):
#       os.makedirs('drive/Colab Notebooks/Gans_out/output2/')
    plt.savefig('drive/My Drive/Colab Notebooks/Gans_out/output2/{}.png'.format(str(c).zfill(3)), bbox_inches='tight')
    c += 1
    plt.close(fig)
    break

arr = [loss_D,loss_G]
pickle.dump(to_out,open("drive/My Drive/Colab Notebooks/Gans_out/Models/loss.dat","wb"))


  